1. Activate the virtual environment: 
source /Users/sofiamorenolasa/Desktop/TFG/.venv/bin/activate
2. Switch to the virtual environment

In [8]:
# Importar pandas
import pandas as pd

# Importar chardet to help with encoding in UTF-8
import chardet

# Path del archivo
file_path = "../data/PARAMETROS_TJ2.csv"

# Read the CSV with ASCII encoding
data = pd.read_csv(file_path, delimiter=";", encoding="ascii")

# Display the first few rows to confirm it loaded successfully
print(data.head())

UnicodeDecodeError: 'ascii' codec can't decode byte 0xd3 in position 15104: ordinal not in range(128)

In [17]:
import pandas as pd

file_path = "../data/PARAMETROS_TJ2.csv"

# Use latin1 encoding
data = pd.read_csv(file_path, delimiter=";", encoding="latin1")

# Display the first few rows
print(data.head())

   N_DESCARGA       fecha   hora comentarioDesc comentarioExp configuracion  \
0         112  19971217.0  19:05            NaN           NaN           NaN   
1         113  19971217.0  19:09            NaN           NaN           NaN   
2         114  19971217.0  19:55            NaN           NaN           NaN   
3         115  19971218.0  11:08            NaN           NaN           NaN   
4         116  19971218.0  11:28            NaN           NaN           NaN   

   potencia_radiada  energia_diamagnetica  retraso_densidad_girotron  zeff  \
0               NaN                   NaN                        NaN   NaN   
1               NaN                   NaN                        NaN   NaN   
2               NaN                   NaN                        NaN   NaN   
3               NaN                   NaN                        NaN   NaN   
4               NaN                   NaN                        NaN   NaN   

   ...  IAccel_nominal_NBI2  tini_NBI2  longitud_pulso_n

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_87239/2676448733.py:6: DtypeWarning: Columns (52,55,96,132,133,144,145) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter=";", encoding="latin1")


Next, we sort the PARAMETROS_TJ2.csv file by the N_DESCARGA field and save the sorted data to a new file named PARAMETROS_TJ2_ORDENADOS.csv. This ensures the rows are ordered starting from the smallest N_DESCARGA value to the largest.

In [14]:
import pandas as pd

# Load the dataset
input_file_path = "../data/PARAMETROS_TJ2.csv"  # Replace with the correct path
output_file_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"

# Read the data
data = pd.read_csv(input_file_path, delimiter=";", encoding="latin1")

# Convert N_DESCARGA to numeric, coercing errors to NaN
data['N_DESCARGA'] = pd.to_numeric(data['N_DESCARGA'], errors='coerce')

# Drop rows where N_DESCARGA is NaN (invalid values)
data = data.dropna(subset=['N_DESCARGA'])

# Ensure N_DESCARGA is an integer
data['N_DESCARGA'] = data['N_DESCARGA'].astype(int)

# Sort by N_DESCARGA in ascending order
data_sorted = data.sort_values(by='N_DESCARGA')

# Save the sorted dataset to a new CSV file
data_sorted.to_csv(output_file_path, index=False, sep=";", encoding="latin1")

print(f"Sorted data saved to {output_file_path}")

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_87239/1307042932.py:8: DtypeWarning: Columns (52,55,96,132,133,144,145) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(input_file_path, delimiter=";", encoding="latin1")


Sorted data saved to ../data/PARAMETROS_TJ2_ORDENADOS.csv


In [12]:
import json

# Initialize an empty list to store JSONL entries
jsonl_data = []

# Example 1: Filter by date range and include comentarioDesc
date_range_start = 20210610
date_range_end = 20210611
filtered_by_date = data[(data["fecha"] >= date_range_start) & (data["fecha"] <= date_range_end)]

prompt_1 = "Dame una lista de los comentarios que había entre el 20210610 y el 20210616."
completion_1 = "\n".join(
    [f"- Descarga {row['N_DESCARGA']}: Comentario: {row['comentarioDesc']}" for _, row in filtered_by_date.iterrows()]
)
jsonl_data.append({"prompt": prompt_1, "completion": completion_1})

# Example 2: Filter by a threshold on a specific column
threshold = 56
filtered_by_threshold = data[data["posicion_sonda_d4top"] > threshold]

prompt_2 = "En qué descargas hubo una posición de la sonda mayor a 50?"
completion_2 = "Descargas: " + ", ".join(map(str, filtered_by_threshold["N_DESCARGA"].tolist()))
jsonl_data.append({"prompt": prompt_2, "completion": completion_2})


# Save the generated training data to a JSONL file
output_path = "../data/llm_training_data.jsonl"
with open(output_path, "w") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"Training data saved to {output_path}")

Training data saved to ../data/llm_training_data.jsonl


In [18]:
import pandas as pd
import json

# Load the data from CSV
data_path = "../data/PARAMETROS_TJ2.csv"
df = pd.read_csv(data_path)

# Replace NaN values with defaults
df = df.fillna({"energia_diamagnetica": "Unknown", "duracion_programada": "Unknown", "duracion_real": "Unknown", "potencia_radiada": 0})

# Initialize JSONL storage
jsonl_data = []

# Example 1: Retrieve comments within a date range
date_range_start = 20210610
date_range_end = 20210611
filtered_by_date = df[(df["fecha"] >= date_range_start) & (df["fecha"] <= date_range_end)]

prompt_1 = "Dame una lista de los comentarios que había entre el 20210610 y el 20210611."
completion_1 = "\n".join(
    [f"- Descarga {row['N_DESCARGA']}: {row['comentarioDesc']}" for _, row in filtered_by_date.iterrows()]
) or "No hay comentarios en este rango de fechas."
jsonl_data.append({"prompt": prompt_1, "completion": completion_1})

# Example 2: Check for gas failures in a specific discharge
discharge_id = 1002  # Replace with a real ID from your data
discharge_data = df[df["N_DESCARGA"] == discharge_id]
if not discharge_data.empty:
    discharge_data = discharge_data.iloc[0]
    prompt_2 = f"¿Hubo fallos de gas durante la descarga {discharge_id}?"
    completion_2 = "Sí, hubo un fallo de gas." if discharge_data["fallo_gas"] else "No, no hubo fallos de gas."
    jsonl_data.append({"prompt": prompt_2, "completion": completion_2})

# Example 3: Compare energy values between two discharges
discharge_a = 1001  # Replace with real IDs
discharge_b = 1002
data_a = df[df["N_DESCARGA"] == discharge_a]
data_b = df[df["N_DESCARGA"] == discharge_b]

if not data_a.empty and not data_b.empty:
    data_a = data_a.iloc[0]
    data_b = data_b.iloc[0]
    prompt_3 = f"¿Qué descarga tuvo más energía diamagnética, la {discharge_a} o la {discharge_b}?"
    if data_a["energia_diamagnetica"] > data_b["energia_diamagnetica"]:
        completion_3 = f"La descarga {discharge_a} tuvo más energía diamagnética ({data_a['energia_diamagnetica']} J)."
    else:
        completion_3 = f"La descarga {discharge_b} tuvo más energía diamagnética ({data_b['energia_diamagnetica']} J)."
    jsonl_data.append({"prompt": prompt_3, "completion": completion_3})

# Example 4: List discharges with radiated power above a threshold
threshold = 300
high_power_discharges = df[df["potencia_radiada"] > threshold]

prompt_4 = f"Dame las descargas con potencia radiada mayor que {threshold}."
completion_4 = "\n".join(
    [f"- Descarga {row['N_DESCARGA']}: Potencia radiada {row['potencia_radiada']} W" for _, row in high_power_discharges.iterrows()]
) or "No hay descargas con potencia radiada mayor que el umbral."
jsonl_data.append({"prompt": prompt_4, "completion": completion_4})

# Example 5: Check the programmed and real durations of a specific discharge
if not discharge_data.empty:
    prompt_5 = f"¿Cuál fue la duración programada y la real de la descarga {discharge_id}?"
    completion_5 = f"La duración programada fue de {discharge_data['duracion_programada']} ms y la real fue de {discharge_data['duracion_real']} ms."
    jsonl_data.append({"prompt": prompt_5, "completion": completion_5})

# Save to JSONL file
output_file = "PARAMETROS_TJ2_training_data.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for entry in jsonl_data:
        json.dump(entry, f, ensure_ascii=False)
        f.write("\n")

print(f"JSONL file saved as {output_file}")

JSONL file saved as PARAMETROS_TJ2_training_data.jsonl
